In [106]:
# Setup pour environnement Docker - Version corrigée
import sys
import os
from pathlib import Path
import json
from dotenv import load_dotenv
from typing import Dict, List, Optional, TypedDict, Annotated

# Charger les variables d'environnement
load_dotenv()

# Configuration avec ta clé OpenRouter
openai_api_key = os.getenv("OPENROUTER_API_KEY")
base_url = "https://openrouter.ai/api/v1"

if not openai_api_key:
    print("❌ Erreur: OPENROUTER_API_KEY non trouvée dans .env")
else:
    print("✅ Clé OpenRouter chargée depuis .env")
    os.environ["OPENAI_API_KEY"] = openai_api_key
    os.environ["OPENAI_API_BASE"] = base_url

print("✅ Setup Docker terminé")

✅ Clé OpenRouter chargée depuis .env
✅ Setup Docker terminé


In [107]:

    from langgraph.graph import StateGraph, END
    from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    from langchain_chroma import Chroma
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.schema import Document
    import chromadb
    import langgraph

In [108]:
    from langchain.embeddings import HuggingFaceEmbeddings
    
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )
    
    # LLM avec OpenRouter (ça marche)
    llm = ChatOpenAI(
        model="openai/gpt-4.1-nano",
        temperature=0.1,
        openai_api_key=openai_api_key,
        openai_api_base=base_url
    )
    
    # Chroma avec embeddings locaux
    vector_store = Chroma(
        collection_name="patent_collection",
        embedding_function=embeddings,
        persist_directory="./chroma_db"
    )

In [109]:
# 1. Charger tous les templates dynamiquement
import os
import json
from langchain.schema import Document

TEMPLATE_DIR = "./"
template_files = [f for f in os.listdir(TEMPLATE_DIR) if f.startswith("template_") and f.endswith(".json")]

docs = []
for fname in template_files:
    with open(os.path.join(TEMPLATE_DIR, fname), "r") as f:
        data = json.load(f)
        docs.append(
            Document(
                page_content=json.dumps(data, indent=2),
                metadata={"source": fname}
            )
        )



In [110]:
# 2. Splitter et vectoriser tous les templates
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(docs)

vector_store = Chroma(
    collection_name="patent_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_db"
)
vector_store.add_documents(split_docs)

# 3. Créer un Retriever avec mémoire de conversation (nouvelle méthode)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [122]:
def print_blue_bold_animated(text):
    """Affiche le texte en bleu gras avec animation lettre par lettre"""
    if not text or str(text).strip() == "":
        print("⚠️ Réponse vide reçue.")
        return
    
    html_start = '<span style="color:#1976d2; font-weight:bold; font-size:1.1em">'
    html_end = '</span>'
    s = ""
    display_id = str(uuid.uuid4())
    
    for c in str(text):
        s += c
        display(HTML(html_start + s + html_end), display_id=display_id, update=True)
        time.sleep(0.002)

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Classe pour stocker l'historique des messages
class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []
    
    def add_messages(self, messages):
        self.messages.extend(messages)
    
    def clear(self):
        self.messages = []

# Créer l'historique
chat_history = InMemoryHistory()

# Template de prompt avec historique
prompt = ChatPromptTemplate.from_messages([
    ("system", "Act as a patent expert. Analyze the provided context and answer each question with precision and expertise, but only if it concerns patents. If a question falls outside the field of patents, clearly state that your expertise is limited to patent-related topics and you are unable to provide assistance otherwise."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    ("system", "Contexte: {context}")
])

# Créer les chaînes
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Ajouter l'historique
conversational_chain = RunnableWithMessageHistory(
    retrieval_chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

# 4. Boucle de chat avec mémoire de conversation
session_id = "session_1"
while True:
    question = input("Pose ta question sur les templates (ou 'quit'): ")
    if question.lower() in ["quit", "exit"]:
        break
    
    result = conversational_chain.invoke(
        {"input": question},
        config={"configurable": {"session_id": session_id}}
    )
    print("Réponse:", result["answer"])
    print("Sources:", [doc.metadata.get("source", "Unknown") for doc in result.get("context", [])])
    print("-" * 50)

Pose ta question sur les templates (ou 'quit'):  "Tell me about drought-resistant crops"


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


Réponse: Based on the provided context, the invention pertains to genetically modified plants with enhanced drought resistance. These modifications involve introducing specific drought-tolerance genes, utilizing technologies such as CRISPR-Cas9, and employing plant transformation protocols. The resulting transgenic plants demonstrate improved water use efficiency, reduced water loss, and sustained crop yields under water-stressed conditions, particularly in arid and semi-arid environments.

If you have specific questions about patentability, patent claims, prior art, or patent application procedures related to drought-resistant crops, I can provide detailed expertise in those areas.
Sources: ['template_biotech_agriculture.json', 'template_biotech_agriculture.json', 'template_biotech_agriculture.json']
--------------------------------------------------
